# Run Batch Inference

Now that we've trained our model, we can run batch inference using the Vertex AI API.

Note that it is not possible to use the Vertex AI SDK, as apparently running batch inference is not yet a feature within that SDK. Meaning we have to use curl.

In [ ]:
import pandas as pd
import json

In [ ]:
# the csv file we're using for testing needs to be in Google Cloud Storage

In [ ]:
!gsutil cp test_data/inference_sample.csv gs://csalling-docai-datasets-regional/beatles/inference_sample.csv

In [ ]:
inference_sample = pd.read_feather("test_data/inference_sample.feather")

In [ ]:
inference_sample.head()

In [ ]:
# parameters
LOCATION = "us-central1"
PROJECT = "mwpmltr"
BATCH_JOB_NAME = "foobar"
MODEL_ID = "projects/mwpmltr/locations/us-central1/models/4377705545990144000"  # I think this is the last successful model that was trained
URI = "gs://csalling-docai-datasets-regional/beatles/inference_sample.csv"
OUTPUT_URI_PREFIX = "gs://csalling-docai-datasets-regional/beatles/batch_prediction_output"
MACHINE_TYPE = "e2-standard-2"
STARTING_REPLICA_COUNT = 1
MAX_REPLICA_COUNT = 10  # default
GENERATE_EXPLANATION = "false"  # default value

In [ ]:
request_body = {
    "displayName": BATCH_JOB_NAME,
    "model": MODEL_ID,
    "inputConfig": {
        "instancesFormat": "csv",
        "gcsSource": {
            "uris": [
                URI
            ]
        },
    },
    "outputConfig": {
        "predictionsFormat": "csv",
        "gcsDestination": {
            "outputUriPrefix": OUTPUT_URI_PREFIX
        }
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
            "acceleratorCount": "0"
        },
        "startingReplicaCount": STARTING_REPLICA_COUNT,
        "maxReplicaCount": MAX_REPLICA_COUNT
    },
    "generateExplanation": GENERATE_EXPLANATION
}

In [ ]:
request_body

In [ ]:
with open("request.json", "w") as outfile:
    outfile.write(json.dumps(request_body))

In [ ]:
api_endpoint = f"https://{LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT}/locations/{LOCATION}/batchPredictionJobs"

In [ ]:
!curl -X POST \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json; charset=utf-8" \
    -d @request.json \
    {api_endpoint}

In [ ]:
!gsutil ls gs://csalling-docai-datasets-regional/beatles/batch_prediction_output/prediction-beatles_automl_file_out_2200_tags-automl-2022_11_29T11_41_37_878Z

In [ ]:
!gsutil cp gs://csalling-docai-datasets-regional/beatles/batch_prediction_output/prediction-beatles_automl_file_out_2200_tags-automl-2022_11_29T11_41_37_878Z/prediction.errors_stats-00000-of-00001.csv errors_stats-00000-of-00001.csv

In [ ]:
pd.read_csv('errors_stats-00000-of-00001.csv')

In [ ]:
!gsutil cp gs://csalling-docai-datasets-regional/beatles/batch_prediction_output/prediction-beatles_automl_file_out_2200_tags-automl-2022_11_29T11_41_37_878Z/prediction.results-00000-of-00002.csv results_1.csv

In [ ]:
!gsutil cp gs://csalling-docai-datasets-regional/beatles/batch_prediction_output/prediction-beatles_automl_file_out_2200_tags-automl-2022_11_29T11_41_37_878Z/prediction.results-00001-of-00002.csv results_2.csv

In [ ]:
results_1 = pd.read_csv('results_1.csv')
results_2 = pd.read_csv('results_2.csv')

In [ ]:
results_1.shape

In [ ]:
results_2.shape

In [ ]:
results_10 = pd.concat([results_1, results_2])

In [ ]:
results_10

In [ ]:
results_10.columns

In [ ]:
results_10.loc[:, 'prediction'] = results_10.Like_The_Beatles_True_scores > results_10.Like_The_Beatles_False_scores

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(results_10.Like_The_Beatles, results_10.prediction)

In [ ]:
confusion_matrix(results_10.Like_The_Beatles, results_10.prediction, normalize='true') * 100

In [ ]:
tn, fp, fn, tp = confusion_matrix(results_10.Like_The_Beatles, results_10.prediction).ravel()

In [ ]:
print(tn)
print(fp)
print(fn)
print(tp)